# TelecomX - Parte 2: Predicción de Churn con Machine Learning

## Misión del Proyecto

¡Bienvenida al equipo de Machine Learning de TelecomX!

Como Analista Junior de Machine Learning, mi misión en este proyecto es desarrollar modelos predictivos robustos para identificar qué clientes tienen mayor probabilidad de cancelar sus servicios. De esta manera, la empresa podrá anticiparse al problema de churn y tomar decisiones estratégicas para retener a sus clientes.

## Objetivos del Challenge

- Preparar los datos para el modelado (tratamiento, codificación, normalización).
- Realizar análisis de correlación y seleccionar las variables más relevantes.
- Entrenar al menos cuatro modelos de clasificación diferentes.
- Evaluar el rendimiento de los modelos utilizando métricas completas.
- Interpretar los resultados y analizar la importancia de las variables.
- Proponer conclusiones estratégicas para la retención de clientes.

> **Nota:** Si es la primera vez que ejecutas este notebook, instala las dependencias necesarias con:
>
> ```
> pip install -r requirements.txt
> ```


In [1]:
# Importación de librerías necesarias
print("Importando librerías para Machine Learning...")

# Verificar e importar librerías básicas
try:
    import pandas as pd
    import numpy as np
    import json
    import warnings
    warnings.filterwarnings('ignore')
    print(" Pandas, NumPy: OK")
except ImportError as e:
    print(f" Error importando librerías básicas: {e}")

# Visualización
try:
    import matplotlib.pyplot as plt
    import seaborn as sns
    print(" Matplotlib, Seaborn: OK")
except ImportError as e:
    print(f" Error: {e}")

try:
    import plotly.express as px
    import plotly.graph_objects as go
    from plotly.subplots import make_subplots
    print(" Plotly: OK")
except ImportError as e:
    print(f" Plotly no disponible: {e}")
    print("Instala con: pip install plotly")

# Machine Learning
try:
    from sklearn.model_selection import train_test_split, cross_val_score
    from sklearn.preprocessing import StandardScaler, LabelEncoder
    from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
    from sklearn.linear_model import LogisticRegression
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.svm import SVC
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.naive_bayes import GaussianNB
    from sklearn.metrics import (
        accuracy_score, precision_score, recall_score, f1_score,
        confusion_matrix, classification_report, roc_auc_score, roc_curve
    )
    from sklearn.utils.class_weight import compute_class_weight
    print(" Scikit-learn: OK")
except ImportError as e:
    print(f"Error importando sklearn: {e}")

# Balanceo de clases (opcional)
try:
    from imblearn.over_sampling import SMOTE
    smote_available = True
    print("SMOTE (imbalanced-learn): OK")
except ImportError:
    smote_available = False
    print("SMOTE no disponible - se usarán class_weights")

# Configuración
plt.style.use('default')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)

print("\n¡Librerías importadas exitosamente!")
print("Listo para entrenar modelos de Machine Learning")


Importando librerías para Machine Learning...
 Pandas, NumPy: OK
 Matplotlib, Seaborn: OK
 Plotly: OK
 Scikit-learn: OK
SMOTE (imbalanced-learn): OK

¡Librerías importadas exitosamente!
Listo para entrenar modelos de Machine Learning


# **PASO 2: Carga de datos**



En esta sección cargaremos los datos desde el archivo JSON y los prepararemos para Machine Learning.


# **PASO 3: Preprocesamiento y selección de variables**

# **PASO 4: División de datos en entrenamiento y prueba**

# **PASO 5: Entrenamiento de modelos**

# **PASO 6: Evaluación de modelos**

# **PASO 7: Análisis de importancia de variables**